In [2]:
from connectPyspark import spark
bc = spark.read.format("avro").load("output/4_CHI_FINAL")
bm = spark.read.format("avro").load("output/4_MALAY_FINAL")
bi =spark.read.format("avro").load("output/4_ENG_FINAL")

bc = bc.coalesce(74)
bm = bm.coalesce(74)
bi = bi.coalesce(74)

:: loading settings :: url = jar:file:/home/pc/TestJupyter/opt/spark-3.3.0/spark-3.3.0-bin-hadoop3/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/pc/.ivy2/cache
The jars for the packages stored in: /home/pc/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5a9a7c46-254e-4f12-a09c-dca58a7707aa;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.3.0 in central
	found org.tukaani#xz;1.8 in central
	found org.spark-project.spark#unused;1.0.0 in central
:: resolution report :: resolve 130ms :: artifacts dl 3ms
	:: modules in use:
	org.apache.spark#spark-avro_2.12;3.3.0 from central in [default]
	org.spark-project.spark#unused;1.0.0 from central in [default]
	org.tukaani#xz;1.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |

22/10/06 02:38:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/06 02:38:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
## non stop word token
import utilities as u
import pyspark.sql.functions as F

[nltk_data] Downloading package stopwords to /home/pc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2022-10-06 02:38:58.033235: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-06 02:38:58.033268: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/pc/TestJupyter/opt/spark-3.3.0/venv-spark/lib/python3.9/site-packages/malaya_boilerplate/frozen_graph.py:35: UserWarning: Cannot import beam_search_ops from Tensorflow Addons, ['malaya.jawi_rumi.deep_model', 'malaya.phoneme.deep_model', 'malaya.rumi_jawi.deep_model', 'malaya.stem.deep_model'] will not available to use, make sure Tensorflow Addons version >= 0.12.0
  warnings.warn(
Using state  server backend.


In [4]:
import json
from nltk.corpus import stopwords
from pyspark.sql.functions import *

malayStopWords = set(json.load(open("/home/pc/Assignment/node_modules/stopwords-ms/stopwords-ms.json")))
stopwordsSet = set(stopwords.words('english') + stopwords.words('chinese') ).union(malayStopWords)

def rem_stop(text):
    token = jieba.cut(text, cut_all=True)
    l = []
    for x in token:
        if x in stopwordsSet or x == "" or x == " ":
            continue
        else:
            l.append(x)
    return l


R_sw = udf(lambda x : rem_stop(x), ArrayType(StringType()))


def get_no_stop_word_df(df, colname):
    return df.withColumn(colname, R_sw(col(colname)))

In [5]:
import utilities as u
from pyspark.sql.functions import array_contains

t = u.get_no_stop_word_df(bi, "text")

y = u.get_no_stop_word_df(bm, "text")

s = u.get_no_stop_word_df(bc, "text")

In [6]:
wc = \
t.rdd.flatMap(lambda x : x.text) \
.map(lambda x : (x,1)).reduceByKey(lambda V1, V2 : V1+V2) \
.toDF(["word","word_count"])

[nltk_data] Downloading package stopwords to /home/pc/nltk_data...(0 + 20) / 20]
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/pc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/pc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/pc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/pc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/pc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/pc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/pc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data

KeyboardInterrupt: 

In [22]:
from pyspark.sql.window import Window

wc\
.select("word","word_count", F.row_number() \
        .over(Window.partitionBy().orderBy(desc(wc['word_count']))) \
        .alias("id")).show(1000)

# 🆂🅰🅼🅿🅻🅴
## CLICK HERE TO SEE TOP ENLISH KEYWORDS

22/10/06 00:48:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/10/06 00:48:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/10/06 00:48:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+--------------+----------+----+
|          word|word_count|  id|
+--------------+----------+----+
|          also|    156098|   1|
|           new|    133806|   2|
|           one|    133231|   3|
|         first|    129372|   4|
|        united|    113027|   5|
|        states|    108925|   6|
|        county|     98812|   7|
|          city|     98727|   8|
|           two|     91922|   9|
|      american|     86644|  10|
|           may|     86492|  11|
|         known|     85162|  12|
|         

In [25]:
wc2 = \
y.rdd.flatMap(lambda x : x.text) \
.map(lambda x : (x,1)).reduceByKey(lambda V1, V2 : V1+V2) \
.toDF(["word","word_count"])

[nltk_data] Downloading package stopwords to /home/pc/nltk_data...(0 + 20) / 20]
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/pc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/pc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/pc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/pc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/pc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/pc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/pc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data

In [27]:
wc2\
.select("word","word_count", F.row_number() \
        .over(Window.partitionBy().orderBy(desc(wc2['word_count']))) \
        .alias("id")).show(1000)



# 🆂🅰🅼🅿🅻🅴
## CLICK HERE TO SEE TOP MALAY KEYWORDS

22/10/06 00:51:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/10/06 00:51:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/10/06 00:51:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+-------------------+----------+----+
|               word|word_count|  id|
+-------------------+----------+----+
|           terletak|    189075|   1|
|              komun|     67626|   2|
|            kampung|     66825|   3|
|             daerah|     53922|   4|
|        perbandaran|     51275|   5|
|            sekolah|     50769|   6|
|           perancis|     44752|   7|
|              filem|     41292|   8|
|         kebangsaan|     40883|   9|
|             google|     34995|  10|
|          

In [9]:
wc3 = \
s.rdd.flatMap(lambda x : x.text) \
.map(lambda x : (x,1)).reduceByKey(lambda V1, V2 : V1+V2) \
.toDF(["word","word_count"])



In [28]:
wc3\
.select("word","word_count", F.row_number() \
        .over(Window.partitionBy().orderBy(desc(wc3['word_count']))) \
        .alias("id")).show(1000)



# 🆂🅰🅼🅿🅻🅴
## CLICK HERE TO SEE TOP CHINESE KEYWORDS

22/10/06 00:52:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/10/06 00:52:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/10/06 00:52:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+--------------+----------+----+
|          word|word_count|  id|
+--------------+----------+----+
|            年|    863100|   1|
|          行星|    717160|   2|
|        年月日|    714212|   3|
|        小行星|    711310|   4|
|          一个|    398627|   5|
|            人|    369625|   6|
|          位于|    315610|   7|
|          中国|    279996|   8|
|          人口|    276993|   9|
|          公里|    276200|  10|
|            中|    244826|  11|
|          平方|    226191|  12|
|          研究|    217106|  13|
| 